Figure(s) in the manuscript created by this notebook: Fig.4C, 3D, 3E.

This notebook fits and plots FRAP data both from clustered proteins and diffuse (unclustered) proteins. The data that this notebook parses comes from the outputs of the "Extract_two_radii_TrackMate.ijm" and "Manual_FRAP_ROI.ijm" ImageJ macros.

In [ ]:
# User-defined parameters for analysis:

# Plotting and figure saving params
save_figs = True
save_dir = '../reports/figures/Fig4C-F_FRAP'
plot_settings = '../src/plotting_settings.py'

# Source data metadata
frame_interval = 1 # in seconds
bleach_n_frames = 8 # how many frames bleaching takes (max)

# Source data location

# Path to diffuse FRAP data (proteins diffusing freely, outside of large clusters)
# Output of "Manual_FRAP_ROI.ijm" ImageJ macro
data_dir_noclust = '../data/processed/Fig4C-F_IRE1_FRAP/manual_ROI_FRAP_non-clustered/intensities'
# Path to cluster FRAP data (text files saved by "Extract_two_radii_TrackMate.ijm")
data_dir = '../data/processed/Fig4C-F_IRE1_FRAP/spot_radii'

# The excluded trace file allows you to manually remove bad traces from the analysis.
# It should be a simple csv with each line containing the file name followed by trace ID
excluded_trace_file = '../data/processed/Fig4C-F_IRE1_FRAP/2018-10-11_excluded_traces.csv'

In [ ]:
# load the built-in and custom modules

# uncomment for debugging
"""
%load_ext autoreload 
%autoreload 2 
"""
import os, sys, inspect
import matplotlib
import matplotlib.pylab as plt
from scipy import stats
import numpy as np
from pprint import pprint
import glob
import pandas as pd
import seaborn as sns


# Add source code directory (src) to path to enable module import
module_dir = '../src'
os.sys.path.insert(0, module_dir)

# import custom modules
import fraptools as frap
import diffusion as dif

In [ ]:
# Set up figure save dirs and load plotting style
if save_figs:
    %matplotlib
    %run $plot_settings save
    
    # Make directories for saving figures
    save_dir_clust = os.path.join(save_dir, 'clusters')
    if not os.path.exists(save_dir_clust):
        os.makedirs(save_dir_clust)
    
    save_dir_diffuse = os.path.join(save_dir, 'diffuse')
    if not os.path.exists(save_dir_diffuse):
        os.makedirs(save_dir_diffuse)
        
    save_dir_summary = os.path.join(save_dir, 'summary')
    if not os.path.exists(save_dir_summary):
        os.makedirs(save_dir_summary)    
        
else:
    %matplotlib inline
    %run $plot_settings plot_only

In [ ]:
# Load FRAP data from clusters

# Populate excluded trace file
if excluded_trace_file:
    excluded_files_df = pd.read_csv(excluded_trace_file)
    excluded_traces = [tuple(x) for x in excluded_files_df.values]
else:
    excluded_traces = (None, None)

# Generate list of all valid cluster FRAP files
frap_files = sorted(glob.glob(os.path.join(data_dir,'*.txt')))

# list of all FRAP data:
frap_data_by_file = []
filenames_no_ext = []

# Go file by file and read data
for file in frap_files:
    
    # Read data from the provided source file
    data = pd.read_csv(file, delimiter='\t')
    filename_no_ext = os.path.split(os.path.splitext(file)[0])[1]
    frap_data_by_file.append(data)
    filenames_no_ext.append(filename_no_ext)

# Extract individual traces from the raw data
df_by_trace, corr_ints, trace_IDs = frap.get_traces_from_df_list(frap_data_by_file, 
                                    filenames_no_ext, exclude=excluded_traces)


In [ ]:
# Load FRAP data from non-clustered proteins

frap_files_noclust = sorted(glob.glob(os.path.join(data_dir_noclust,'*.csv')))

# list of all FRAP data:
frap_data_by_file_noclust = []
filenames_no_ext_noclust = []

# Go file by file and read data
for file in frap_files_noclust:
    
    # Read data from the provided source file
    data = pd.read_csv(file, delimiter=',')
    filename_no_ext = os.path.split(os.path.splitext(file)[0])[1]
    frap_data_by_file_noclust.append(data)
    filenames_no_ext_noclust.append(filename_no_ext)

# break up data into smaller data frames, one per trace
df_by_trace_noclust, corr_ints_noclust, trace_IDs_noclust = \
    frap.read_nonclust_frap_data(frap_data_by_file_noclust, 
                        filenames_no_ext_noclust, exclude=excluded_traces)

In [ ]:
# Analyze and plot the FRAP data from clusters

# Fit the individual FRAP traces
fit, data = frap.fit_frap_smart(corr_ints, frame_interval, bleach_n_frames)

# Plot results
for f,d,trace_ID in zip(fit, data, trace_IDs):
    
    file_name = trace_ID[0]
    trace_num = trace_ID[1]
    
    full_name = file_name + '_trace-ID_' + str(trace_num)
    
    fig, axarr = frap.plot_fit_results(f,d)
    fig.canvas.set_window_title(full_name)
    plt.suptitle(full_name)
    if save_figs:
        fig_filename_pdf = os.path.join(save_dir_clust, (full_name+'.pdf'))
        plt.savefig(fig_filename_pdf)
        plt.close(fig)
print("done")

In [ ]:
# Analyze and plot FRAP data from diffuse (non-clustered) proteins

# Fit the individual FRAP traces
fit_noclust, data_noclust = frap.fit_frap_smart(corr_ints_noclust, frame_interval, bleach_n_frames)

# Plot results
for f,d,trace_ID in zip(fit_noclust, data_noclust, trace_IDs_noclust):
    
    file_name = trace_ID[0]
    trace_num = trace_ID[1]
    
    full_name = file_name + '_trace-ID_' + str(trace_num)
    
    fig, axarr = frap.plot_fit_results(f,d)
    fig.canvas.set_window_title(full_name)
    plt.suptitle(full_name)
    if save_figs:
        fig_filename_pdf = os.path.join(save_dir_diffuse, (full_name+'.pdf'))
        plt.savefig(fig_filename_pdf)
        plt.close()
        print("Processed trace ID", trace_ID)
print("done")

In [ ]:
# Summarize fit results

# Prepare data for plotting
frap_fits_foci = fit
thalf_foci = [f['thalf'] for f in frap_fits_foci]
mobile_f_foci = [f['mobile_fraction'] for f in frap_fits_foci]   

thalf_noclust = [f['thalf'] for f in fit_noclust]
mobile_f_noclust = [f['mobile_fraction'] for f in fit_noclust]   

print(np.mean(thalf_foci), np.mean(mobile_f_foci), stats.sem(mobile_f_foci))
print(np.mean(thalf_noclust), np.mean(mobile_f_noclust))

print('t-test for thalf: ', stats.ttest_ind(thalf_foci, thalf_noclust))

# Create summary figure
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(2.6, 1.7))
fig.tight_layout(pad=2)

df1 = pd.DataFrame({'Free' : thalf_noclust})
df2 = pd.DataFrame({'Clustered' : thalf_foci})
df_thalf = pd.concat([df1, df2], axis = 1)

df3 = pd.DataFrame({'Free' : mobile_f_noclust})
df4 = pd.DataFrame({'Clustered' : mobile_f_foci})
df_mobile = pd.concat([df3, df4], axis = 1)

sns.boxplot(data=df_thalf, linewidth=0.5, showfliers = False, 
            boxprops={'facecolor':'None'}, ax=axes[0])
sns.swarmplot(data=df_thalf, zorder=0.5, ax=axes[0], size=2)
axes[0].set_ylabel('Half-time of recovery (s)')

sns.boxplot(data=df_mobile, linewidth=0.5, showfliers = False,
            boxprops={'facecolor':'None'}, ax=axes[1])
sns.swarmplot(data=df_mobile, zorder=0.5, ax=axes[1], size=2)
axes[1].set_ylabel('Mobile fraction')


if save_figs:
    fig_filename_pdf = os.path.join(save_dir_summary, 'Clusters_vs_noClusters_recovery.pdf')
    plt.savefig(fig_filename_pdf)

plt.show()

In [ ]:
# Estimate diffusion constant (from doi:10.1111/tra.12008)

rn = 5 # In microns, effective radius of the bleach spot (st.dev. of the Gaussian beam)

d_foci = [0.25*rn**2/t for t in thalf_foci]
d_er = [0.25*rn**2/t for t in thalf_noclust]

print(np.mean(d_foci), stats.sem(d_foci))
print(np.mean(d_er), stats.sem(d_er))


# Estimate diffusion using Guigas-Weiss model
T = 310 # 37C in Kelvin
c = 6 # in nm; 4 for slip, 6 for stick boundary conditions
ire1_radius = 0.7 # estimated in-plane radius of IRE1 in nm


print('Estimated IRE1 D, um^2/s: ', dif.diffconst_gw(ire1_radius,T,c))
print('Saffman-Delbruck IRE1 D, um^2/s: ', dif.diffconst_sd(ire1_radius,T,c))
